In [60]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [61]:
base = pd.read_csv('datasets/location_output.csv')
base['NEIGHBORHOOD'] = base.NEIGHBORHOOD.str.strip()
base['NEIGHBORHOOD'] = base.NEIGHBORHOOD.str.lower()
base['NEIGHBORHOOD'] = base.NEIGHBORHOOD.str.replace('-', ' ')
base['NEIGHBORHOOD'] = base.NEIGHBORHOOD.str.replace('   ', ' ')
display(base)

,Location_key,ADDRESS,LAT,LON,NEIGHBORHOOD
0,0,9XX TERMINAL AVE,49.269802,-123.083763,strathcona
1,1,63XX WILTSHIRE ST,49.228051,-123.146610,kerrisdale
2,2,40XX W 19TH AVE,49.255559,-123.193725,dunbar southlands
3,3,18XX E 3RD AVE,49.267734,-123.067654,grandview woodland
4,4,40XX W 21ST AVE,49.253762,-123.194407,dunbar southlands
5,5,18XX E 3RD AVE,49.267814,-123.067441,grandview woodland
6,6,18XX E 3RD AVE,49.267731,-123.067302,grandview woodland
7,7,63XX WINDSOR ST,49.226430,-123.085283,sunset
8,8,10XX ALBERNI ST,49.284715,-123.122824,west end
9,9,18XX E 3RD AVE,49.267811,-123.067089,grandview woodland


In [62]:
stats = pd.read_csv('datasets/nhbdStats.csv')
stats['NEIGHBORHOOD'] = stats.NEIGHBORHOOD.str.strip()
stats['NEIGHBORHOOD'] = stats.NEIGHBORHOOD.str.lower()
stats['NEIGHBORHOOD'] = stats.NEIGHBORHOOD.str.replace('-', ' ')
stats['NEIGHBORHOOD'] = stats.NEIGHBORHOOD.str.replace('   ', ' ')
display(stats)

,NEIGHBORHOOD,TTL_POPULATION_ALL,PCT_HISPANIC,PCT_WHITE,PCT_BLACK,PCT_NATIVEAM,PCT_ASIAN,PCT_HAWAIIANPI,PCT_OTHERRACE,MALE,...,AGE_20_TO_29,AGE_30_TO_39,AGE_40_TO_49,AGE_50_TO_59,AGE_60_OVER,MEDIAN_AGE_ALL,MEDIAN_AGE_MALE,MEDIAN_AGE_FEMALE,PER_CAPITA_INCOME,PCT_LOW_INCOME
0,arbutus ridge,15295.0,0.497512,41.890547,0.298507,0.928690,55.522388,0.000000,1.658375,6925.0,...,1970.0,1130.0,2315.0,2230.0,4415.0,46.200000,44.400000,47.400000,30929.0,24.400000
1,downtown,62030.0,2.693059,56.341857,1.112905,2.429700,36.420015,0.000000,1.970946,33100.0,...,12860.0,16370.0,9365.0,8130.0,10125.0,37.100000,38.100000,36.100000,41858.0,23.800000
2,dunbar southlands,21425.0,0.822175,61.146347,0.563777,4.134367,36.269673,0.000000,0.963120,10340.0,...,2850.0,1460.0,3350.0,3355.0,5060.0,44.100000,42.800000,44.900000,40463.0,15.900000
3,fairview,33620.0,1.482047,74.056532,1.161192,1.802903,21.527884,0.000000,1.329259,15115.0,...,5890.0,7570.0,4790.0,4255.0,7875.0,40.200000,39.100000,41.400000,46940.0,14.700000
4,grandview woodland,29175.0,1.568695,71.970350,1.741079,7.791760,23.030512,0.000000,1.361834,14255.0,...,4975.0,6555.0,4645.0,3910.0,5050.0,38.100000,38.100000,38.100000,32438.0,20.500000
5,hastings sunrise,34575.0,2.154478,41.316137,0.850066,3.224388,53.187747,0.000000,2.227759,16910.0,...,5115.0,4780.0,4915.0,5290.0,8095.0,42.100000,41.000000,43.100000,27255.0,16.900000
6,kensington cedar cottage,49325.0,1.503990,36.320851,1.012891,2.056476,58.532842,0.000000,2.475957,24060.0,...,7795.0,8010.0,7340.0,6760.0,9715.0,38.800000,37.600000,39.900000,28356.0,14.800000
7,kerrisdale,13975.0,0.755668,41.525729,0.143937,0.503778,55.811443,0.000000,1.475351,6495.0,...,2055.0,1105.0,2015.0,2290.0,3585.0,45.600000,44.200000,46.400000,35064.0,22.000000
8,killarney,29325.0,2.367784,28.482544,1.555479,1.469063,64.258555,0.000000,3.076391,13935.0,...,3750.0,3610.0,4205.0,4685.0,7125.0,43.400000,41.300000,45.000000,29259.0,17.000000
9,kitsilano,43045.0,1.637235,79.932172,0.935563,1.730792,16.009823,0.000000,1.192843,20115.0,...,8550.0,8720.0,6195.0,5515.0,8330.0,37.700000,37.200000,38.200000,44084.0,14.700000


In [63]:
location = pd.merge(base, stats, left_on='NEIGHBORHOOD', right_on='NEIGHBORHOOD', how='left')
display(location)

,Location_key,ADDRESS,LAT,LON,NEIGHBORHOOD,TTL_POPULATION_ALL,PCT_HISPANIC,PCT_WHITE,PCT_BLACK,PCT_NATIVEAM,...,AGE_20_TO_29,AGE_30_TO_39,AGE_40_TO_49,AGE_50_TO_59,AGE_60_OVER,MEDIAN_AGE_ALL,MEDIAN_AGE_MALE,MEDIAN_AGE_FEMALE,PER_CAPITA_INCOME,PCT_LOW_INCOME
0,0,9XX TERMINAL AVE,49.269802,-123.083763,strathcona,12585.0,1.725013,54.794521,2.993404,10.147133,...,1460.0,1920.0,1815.0,2230.0,3720.0,48.300000,50.000000,45.400000,17631.0,49.700000
1,1,63XX WILTSHIRE ST,49.228051,-123.146610,kerrisdale,13975.0,0.755668,41.525729,0.143937,0.503778,...,2055.0,1105.0,2015.0,2290.0,3585.0,45.600000,44.200000,46.400000,35064.0,22.000000
2,2,40XX W 19TH AVE,49.255559,-123.193725,dunbar southlands,21425.0,0.822175,61.146347,0.563777,4.134367,...,2850.0,1460.0,3350.0,3355.0,5060.0,44.100000,42.800000,44.900000,40463.0,15.900000
3,3,18XX E 3RD AVE,49.267734,-123.067654,grandview woodland,29175.0,1.568695,71.970350,1.741079,7.791760,...,4975.0,6555.0,4645.0,3910.0,5050.0,38.100000,38.100000,38.100000,32438.0,20.500000
4,4,40XX W 21ST AVE,49.253762,-123.194407,dunbar southlands,21425.0,0.822175,61.146347,0.563777,4.134367,...,2850.0,1460.0,3350.0,3355.0,5060.0,44.100000,42.800000,44.900000,40463.0,15.900000
5,5,18XX E 3RD AVE,49.267814,-123.067441,grandview woodland,29175.0,1.568695,71.970350,1.741079,7.791760,...,4975.0,6555.0,4645.0,3910.0,5050.0,38.100000,38.100000,38.100000,32438.0,20.500000
6,6,18XX E 3RD AVE,49.267731,-123.067302,grandview woodland,29175.0,1.568695,71.970350,1.741079,7.791760,...,4975.0,6555.0,4645.0,3910.0,5050.0,38.100000,38.100000,38.100000,32438.0,20.500000
7,7,63XX WINDSOR ST,49.226430,-123.085283,sunset,36500.0,1.926542,17.144837,0.970201,1.469161,...,5875.0,4985.0,5200.0,4930.0,7475.0,38.700000,36.800000,40.400000,25498.0,16.000000
8,8,10XX ALBERNI ST,49.284715,-123.122824,west end,47200.0,3.820634,71.350599,1.337757,2.290240,...,10475.0,11890.0,6425.0,6175.0,9680.0,38.400000,39.200000,37.400000,36425.0,23.300000
9,9,18XX E 3RD AVE,49.267811,-123.067089,grandview woodland,29175.0,1.568695,71.970350,1.741079,7.791760,...,4975.0,6555.0,4645.0,3910.0,5050.0,38.100000,38.100000,38.100000,32438.0,20.500000


In [64]:
location.loc[np.isnan(location['TTL_POPULATION_ALL'])]

,Location_key,ADDRESS,LAT,LON,NEIGHBORHOOD,TTL_POPULATION_ALL,PCT_HISPANIC,PCT_WHITE,PCT_BLACK,PCT_NATIVEAM,...,AGE_20_TO_29,AGE_30_TO_39,AGE_40_TO_49,AGE_50_TO_59,AGE_60_OVER,MEDIAN_AGE_ALL,MEDIAN_AGE_MALE,MEDIAN_AGE_FEMALE,PER_CAPITA_INCOME,PCT_LOW_INCOME
10,10,OFFSET TO PROTECT PRIVACY,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,9XX SEYMOUR ST,49.279374,-123.120920,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,12,9XX SEYMOUR ST,49.278701,-123.121770,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,13,9XX ROBSON ST,49.282692,-123.121932,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,19,11XX MAINLAND ST,49.274721,-123.121745,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,20,11XX HAMILTON ST,49.275844,-123.121104,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,25,9XX DAVIE ST,49.278435,-123.128270,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,31,9XX CANADA PL,49.288110,-123.115333,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,41,11XX MAINLAND ST,49.274644,-123.121863,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,42,11XX MAINLAND ST,49.274567,-123.121982,central business district,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
location.to_csv('datasets/final/location.csv')

In [66]:
location.columns

Index(['Location_key', 'ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD',
       'TTL_POPULATION_ALL', 'PCT_HISPANIC', 'PCT_WHITE', 'PCT_BLACK',
       'PCT_NATIVEAM', 'PCT_ASIAN', 'PCT_HAWAIIANPI', 'PCT_OTHERRACE', 'MALE',
       'FEMALE', 'AGE_10_TO_19', 'AGE_20_TO_29', 'AGE_30_TO_39',
       'AGE_40_TO_49', 'AGE_50_TO_59', 'AGE_60_OVER', 'MEDIAN_AGE_ALL',
       'MEDIAN_AGE_MALE', 'MEDIAN_AGE_FEMALE', 'PER_CAPITA_INCOME',
       'PCT_LOW_INCOME'],
      dtype='object')